# STFT Dataset Generation

In [3]:
import time
import os
import pprint
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import IPython.display as ipd
from IPython.display import Audio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa
import librosa.display
import utils

print(librosa.__version__)

0.9.2


In [4]:
AUDIO_DIR = os.environ.get('AUDIO_DIR')
print("audio directory: ",AUDIO_DIR)
print("Loading tracks.csv...")
tracks = utils.load('data/fma_metadata/tracks.csv')
#get only the small subset of the dataset
small = tracks[tracks['set', 'subset'] <= 'small']
print("small dataset shape:",small.shape)

audio directory:  ./data/fma_small/
Loading tracks.csv...
small dataset shape: (8000, 52)


In [5]:
#split into train, validation and test sets
train = small.index[small['set', 'split'] == 'training']
val = small.index[small['set', 'split'] == 'validation']
test = small.index[small['set', 'split'] == 'test']

print('{} training examples, {} validation examples, {} testing examples'.format(*map(len, [train, val, test])))


6400 training examples, 800 validation examples, 800 testing examples


In [6]:
genres=utils.load('data/fma_metadata/genres.csv')
#list of genres
genres_list=genres['title'].unique()

In [7]:
def expand_labels(labels_set):
    lbl_array = labels_set

    # Repeat each element 10 times using np.repeat()
    expanded_array = np.repeat(lbl_array, 10)

    # Create a new Categorical object from the expanded array
    expanded_categorical = pd.Categorical(expanded_array)

    # Print the expanded categorical variable
    return expanded_categorical

In [8]:
#retrieve labels for each subset
tr_labels = small.loc[small[('set', 'split')] == 'training', ('track', 'genre_top')].values
vl_labels = small.loc[small[('set', 'split')] == 'validation', ('track', 'genre_top')].values
ts_labels = small.loc[small[('set', 'split')] == 'test', ('track', 'genre_top')].values

tr_labels=expand_labels(tr_labels)
vl_labels=expand_labels(vl_labels)
ts_labels=expand_labels(ts_labels)


print('{} training labels, {} validation labels, {} testing labels'.format(*map(len, [tr_labels, vl_labels, ts_labels])))


64000 training labels, 8000 validation labels, 8000 testing labels


## Generate the STFT small dataset

### Let's get all the STFT spectrograms for each track audio in the dataset

Some tracks (6) of the original small dataset are corrupted or very small (1-2 seconds), these track has been replaced
by a script with other good tracks of the same genre from the dataset.

#### Important
Create a directory 'fma_small_stft' with 3 directories inside:
- 'train'
- 'validation'
- 'test'

In [98]:
def generate_stft_files(dataset,directory):
    print("Generating files in the folder:",directory,"...")

    track_ids = np.array(dataset)
    
    error_list = [] #list of tracks which caused errors
    progress_bar = 1
    for track_id in track_ids:
        #print("track id: ",track_id)
        j = 0
        try:
            #get the filename using the utils
            filename = utils.get_audio_path(AUDIO_DIR, track_id)

            x, sr = librosa.load(filename, sr=22050, mono=True) #load the MONO audio file from the data/fma_small directory
            print('File: {}, Duration: {:.2f}s, {} samples, sampling rate: {}'.format(filename,x.shape[-1] / sr, x.size, sr))
            
            #calculate stft on each 3s clip of the 30s song (using 50% clip overlapping we have a total of 20 clips per song)  
            clip_startings = np.arange(0,30,1.5)
            #print("a total of",len(clip_startings),"clips have been created from the song, clip startings:",clip_startings)
                  
            for start in clip_startings:
                end = start + 3 #start and end point of the clip
                #print("clip number: {}, start: {}, end: {}".format(j,start,end))
                save_filename = './data/fma_small_stft_transposed_22050_overlapped/' + directory + "/" +  str(track_id) + "_" + str(j)
                j+=1 # for the filename
                
                start_sample = int(start*sr)
                end_sample = int(end*sr)
                #print("start_sample:",start_sample,"end_sample:",end_sample)
                if(start == 28.5):
                    #print("This is the last clip")
                    clip = np.append(x[start_sample:], [0 for i in range(int(1.5*sr))])
                    #print("padded clip:",clip)
                else:
                    clip = x[start_sample:end_sample] #extract the clip
                #print("samples per clip:",len(clip),"duration in seconds = ",len(clip)/sr)
                

                
                #calculate stft using a window of 1024 sample and a hop length of 1024 sample
                stft = librosa.stft(clip, n_fft=1024, hop_length=512)
                #take only the absolute value of the stft
                stft = np.abs(stft)
                #print("stft shape: {}".format(stft.shape))
                #transpose the stft
                stft = stft.T
                #print("stft shape (after transpose): {}".format(stft.shape))

                #resize the spectrogram to become 128x513 (adds padding with zeros if too short)
                stft = librosa.util.fix_length(stft, size=128, axis=0)[:, :513]

                #stft = stft[:128]
                #print("stft shape (after fixing length to 128 frames): {}".format(stft.shape))

                #print("Saving the stft vector in file:",save_filename) 
                np.save(save_filename,stft)
                #print("shape of the stft vector (clip starting at " + str(start) + "): ", stft.shape)
           
        except Exception as e: #skip the songs which give error (file corrupted or song too short)
            print("ERROR on file: ",filename,"\nError information:", e)
            error_list.append(filename)
            
        print("Progress:" + str(progress_bar) + "/" + str(len(track_ids)))
        progress_bar +=1
        if(len(error_list)>0):
            print("***ERROR: {} tracks caused errors: {}".format(len(error_list),error_list))
        #else:
            #print("Procedure completed without errors.")
            
    print("\n\n###Procedure completed with {} errors: {}".format(len(error_list),error_list))


In [99]:
generate_stft_files(train,'train')

Generating files in the folder: train ...
File: ./data/fma_small/000/000002.mp3, Duration: 29.98s, 660984 samples, sampling rate: 22050
Progress:1/6400
File: ./data/fma_small/000/000005.mp3, Duration: 30.00s, 661560 samples, sampling rate: 22050
Progress:2/6400
File: ./data/fma_small/000/000010.mp3, Duration: 29.98s, 660984 samples, sampling rate: 22050
Progress:3/6400
File: ./data/fma_small/000/000140.mp3, Duration: 29.98s, 660984 samples, sampling rate: 22050
Progress:4/6400
File: ./data/fma_small/000/000141.mp3, Duration: 29.98s, 660984 samples, sampling rate: 22050
Progress:5/6400
File: ./data/fma_small/000/000190.mp3, Duration: 30.00s, 661560 samples, sampling rate: 22050
Progress:6/6400
File: ./data/fma_small/000/000193.mp3, Duration: 29.98s, 660984 samples, sampling rate: 22050
Progress:7/6400
File: ./data/fma_small/000/000194.mp3, Duration: 30.00s, 661560 samples, sampling rate: 22050
Progress:8/6400
File: ./data/fma_small/000/000197.mp3, Duration: 29.98s, 660984 samples, sampl

KeyboardInterrupt: 

In [89]:
generate_stft_files(val,'validation')

Generating files in the folder: validation ...
track id:  148
File: ./data/fma_small/000/000148.mp3
Duration: 29.98s, 660984 samples, sampling rate: 22050
a total of 20 clips have been created from the song, clip startings: [ 0.   1.5  3.   4.5  6.   7.5  9.  10.5 12.  13.5 15.  16.5 18.  19.5
 21.  22.5 24.  25.5 27.  28.5]
clip number: 0, start: 0.0, end: 3.0
start_sample: 0 end_sample: 66150
samples per clip: 66150 duration in seconds =  3.0
stft shape: (513, 130)
stft shape (after transpose): (130, 513)
stft shape (after fixing length to 128 frames): (128, 513)
Saving the stft vector in file: ./data/fma_small_stft_transposed_22050_overlapped/validation/148_0
shape of the stft vector (clip starting at 0.0):  (128, 513)
clip number: 1, start: 1.5, end: 4.5
start_sample: 33075 end_sample: 99225
samples per clip: 66150 duration in seconds =  3.0
stft shape: (513, 130)
stft shape (after transpose): (130, 513)
stft shape (after fixing length to 128 frames): (128, 513)
Saving the stft vec

Duration: 30.00s, 661560 samples, sampling rate: 22050
a total of 20 clips have been created from the song, clip startings: [ 0.   1.5  3.   4.5  6.   7.5  9.  10.5 12.  13.5 15.  16.5 18.  19.5
 21.  22.5 24.  25.5 27.  28.5]
clip number: 0, start: 0.0, end: 3.0
start_sample: 0 end_sample: 66150
samples per clip: 66150 duration in seconds =  3.0
stft shape: (513, 130)
stft shape (after transpose): (130, 513)
stft shape (after fixing length to 128 frames): (128, 513)
Saving the stft vector in file: ./data/fma_small_stft_transposed_22050_overlapped/validation/459_0
shape of the stft vector (clip starting at 0.0):  (128, 513)
clip number: 1, start: 1.5, end: 4.5
start_sample: 33075 end_sample: 99225
samples per clip: 66150 duration in seconds =  3.0
stft shape: (513, 130)
stft shape (after transpose): (130, 513)
stft shape (after fixing length to 128 frames): (128, 513)
Saving the stft vector in file: ./data/fma_small_stft_transposed_22050_overlapped/validation/459_1
shape of the stft ve

Duration: 29.98s, 660984 samples, sampling rate: 22050
a total of 20 clips have been created from the song, clip startings: [ 0.   1.5  3.   4.5  6.   7.5  9.  10.5 12.  13.5 15.  16.5 18.  19.5
 21.  22.5 24.  25.5 27.  28.5]
clip number: 0, start: 0.0, end: 3.0
start_sample: 0 end_sample: 66150
samples per clip: 66150 duration in seconds =  3.0
stft shape: (513, 130)
stft shape (after transpose): (130, 513)
stft shape (after fixing length to 128 frames): (128, 513)
Saving the stft vector in file: ./data/fma_small_stft_transposed_22050_overlapped/validation/602_0
shape of the stft vector (clip starting at 0.0):  (128, 513)
clip number: 1, start: 1.5, end: 4.5
start_sample: 33075 end_sample: 99225
samples per clip: 66150 duration in seconds =  3.0
stft shape: (513, 130)
stft shape (after transpose): (130, 513)
stft shape (after fixing length to 128 frames): (128, 513)
Saving the stft vector in file: ./data/fma_small_stft_transposed_22050_overlapped/validation/602_1
shape of the stft ve

Duration: 29.98s, 660984 samples, sampling rate: 22050
a total of 20 clips have been created from the song, clip startings: [ 0.   1.5  3.   4.5  6.   7.5  9.  10.5 12.  13.5 15.  16.5 18.  19.5
 21.  22.5 24.  25.5 27.  28.5]
clip number: 0, start: 0.0, end: 3.0
start_sample: 0 end_sample: 66150
samples per clip: 66150 duration in seconds =  3.0
stft shape: (513, 130)
stft shape (after transpose): (130, 513)
stft shape (after fixing length to 128 frames): (128, 513)
Saving the stft vector in file: ./data/fma_small_stft_transposed_22050_overlapped/validation/690_0
shape of the stft vector (clip starting at 0.0):  (128, 513)
clip number: 1, start: 1.5, end: 4.5
start_sample: 33075 end_sample: 99225
samples per clip: 66150 duration in seconds =  3.0
stft shape: (513, 130)
stft shape (after transpose): (130, 513)
stft shape (after fixing length to 128 frames): (128, 513)
Saving the stft vector in file: ./data/fma_small_stft_transposed_22050_overlapped/validation/690_1
shape of the stft ve

Duration: 30.00s, 661560 samples, sampling rate: 22050
a total of 20 clips have been created from the song, clip startings: [ 0.   1.5  3.   4.5  6.   7.5  9.  10.5 12.  13.5 15.  16.5 18.  19.5
 21.  22.5 24.  25.5 27.  28.5]
clip number: 0, start: 0.0, end: 3.0
start_sample: 0 end_sample: 66150
samples per clip: 66150 duration in seconds =  3.0
stft shape: (513, 130)
stft shape (after transpose): (130, 513)
stft shape (after fixing length to 128 frames): (128, 513)
Saving the stft vector in file: ./data/fma_small_stft_transposed_22050_overlapped/validation/993_0
shape of the stft vector (clip starting at 0.0):  (128, 513)
clip number: 1, start: 1.5, end: 4.5
start_sample: 33075 end_sample: 99225
samples per clip: 66150 duration in seconds =  3.0
stft shape: (513, 130)
stft shape (after transpose): (130, 513)
stft shape (after fixing length to 128 frames): (128, 513)
Saving the stft vector in file: ./data/fma_small_stft_transposed_22050_overlapped/validation/993_1
shape of the stft ve

KeyboardInterrupt: 

In [90]:
generate_stft_files(test,'test')

Generating files in the folder: test ...
track id:  182
File: ./data/fma_small/000/000182.mp3
Duration: 30.00s, 661560 samples, sampling rate: 22050
a total of 20 clips have been created from the song, clip startings: [ 0.   1.5  3.   4.5  6.   7.5  9.  10.5 12.  13.5 15.  16.5 18.  19.5
 21.  22.5 24.  25.5 27.  28.5]
clip number: 0, start: 0.0, end: 3.0
start_sample: 0 end_sample: 66150
samples per clip: 66150 duration in seconds =  3.0
stft shape: (513, 130)
stft shape (after transpose): (130, 513)
stft shape (after fixing length to 128 frames): (128, 513)
Saving the stft vector in file: ./data/fma_small_stft_transposed_22050_overlapped/test/182_0
shape of the stft vector (clip starting at 0.0):  (128, 513)
clip number: 1, start: 1.5, end: 4.5
start_sample: 33075 end_sample: 99225
samples per clip: 66150 duration in seconds =  3.0
stft shape: (513, 130)
stft shape (after transpose): (130, 513)
stft shape (after fixing length to 128 frames): (128, 513)
Saving the stft vector in file:

KeyboardInterrupt: 

## Generate the raw audio splitted dataset
Split each audio track in 3s clips with the name "track_id_clip_id" (e.g "2_0", "2_1", ... , "2_9") and save it as .mp3

- create a folder "fma_small_raw_split" with folders inside:
    - train
    - validation
    - test

In [9]:
import soundfile as sf

def generate_raw_files(dataset,directory):
    print("Generating raw files in the folder:",directory,"...")

    track_ids = np.array(dataset)
    
    error_list = [] #list of tracks which caused errors
    progress_bar = 1
    for track_id in track_ids:
        #rint("track id: ",track_id)

        try:
            #get the filename using the utils
            filename = utils.get_audio_path(AUDIO_DIR, track_id)
            print('File: {}'.format(filename))

            x, sr = librosa.load(filename, sr=44100, mono=True) #load the MONO audio file from the data/fma_small directory
            #print('Duration: {:.2f}s, {} samples, sampling rate: {}'.format(x.shape[-1] / sr, x.size, sr))
            total_length = 44100 * 30
            #resize the audio to have a length of  total_length_sample(adds padding with zeros if too short)
            x = librosa.util.fix_length(x, size=total_length)[:total_length]
            #print("sampling rate:",sr)
            
            #calculate stft on each 3s clip of the 30s song
            for i in range (0,28,3):
                start, end = i, i+3 #start and end point of the clip
                #print("start: {}, end: {}".format(start,end))
                clip = x[start*sr:end*sr] #extract the clip     
                save_filename = './data/fma_small_raw_split/' + directory + '/' +  str(track_id) + "_" + str(i//3) + '.mp3'
                #print("Saving the clip vector in file:",save_filename, "shape:",clip.shape) 
                sf.write(save_filename, clip, 44100, format='mp3')    

                #print("shape of the stft vector (clip " + str(i//3) + "): ", stft.shape)
                
        except Exception as e: #skip the songs which give error (file corrupted or song too short)
            print("ERROR on file: ",filename,"\nError information:", e)
            error_list.append(filename)
            
        print("Progress:" + str(progress_bar) + "/" + str(len(track_ids)))
        progress_bar +=1
        if(len(error_list)>0):
            print("***ERROR: {} tracks caused errors: {}".format(len(error_list),error_list))
            
    print("\n\n###Procedure completed with {} errors: {}".format(len(error_list),error_list))


In [10]:
generate_raw_files(train, 'train')

Generating raw files in the folder: train ...
File: ./data/fma_small/000/000002.mp3
Progress:1/6400
File: ./data/fma_small/000/000005.mp3
Progress:2/6400
File: ./data/fma_small/000/000010.mp3
Progress:3/6400
File: ./data/fma_small/000/000140.mp3
Progress:4/6400
File: ./data/fma_small/000/000141.mp3
Progress:5/6400
File: ./data/fma_small/000/000190.mp3


KeyboardInterrupt: 

In [ ]:
track = np.load('./data/fma_small_raw_split/train/2_0.npy')
print("shape:",track.shape)

In [ ]:
generate_raw_files(val, 'validation')

In [ ]:
generate_raw_files(test, 'test')

In [11]:
from pydub import AudioSegment

In [85]:
def generate_raw_array_files(dataset,directory):
    print("Generating files in the folder:",directory,"...")
    # Desired length in milliseconds
    desired_length_ms = 30000
    sample_width = 2  # 16-bit sample width
    channels = 1  # Mono
    frame_rate = 22050  # 22050 kHz sampling rate
    total_length_sample = int(desired_length_ms*frame_rate/1000) #
    
    track_ids = np.array(dataset)
    
    
    
    
    error_list = [] #list of tracks which caused errors
    progress_bar = 1
    for track_id in track_ids:
        #print("track id: ",track_id)

        try:
            #get the filename using the utils
            filename = utils.get_audio_path(AUDIO_DIR, track_id)
            print("File: ",filename,",Progress:" + str(progress_bar) + "/" + str(len(track_ids)))

            # Load the audio files
            audio = AudioSegment.from_file(filename)

            audio = audio.set_frame_rate(frame_rate).set_channels(channels).set_sample_width(sample_width)

            # Trim or zero-pad audio clip to the desired length
            audio = audio[:desired_length_ms] if len(audio) > desired_length_ms else audio + AudioSegment.silent(duration=desired_length_ms - len(audio))
            
            # Convert to raw PCM arrays
            pcm_audio = np.frombuffer(audio.raw_data, dtype=np.int16)
            #print("pcm_audio: shape:",pcm_audio.shape,"content:",pcm_audio,"type:",pcm_audio.dtype)
            
            if pcm_audio.shape[0]<total_length_sample: #if audio is less than 30sec, pad with trailing zeros
                #print("padding needed because:pcm_audio.shape: ",pcm_audio.shape)
                padding = np.zeros(total_length_sample-pcm_audio.shape[0], dtype=np.int16)
                #print("padding: content:",padding,"shape:",padding.shape)
                pcm_audio = np.append(pcm_audio,padding)
                #print("padded audio: shape: ",pcm_audio.shape,"type:",pcm_audio.dtype)
                
             
            #split the vector into 20 50% overlapping clips of 3s (last clip is padded to 0)
            #print("pcm_audio: shape:",pcm_audio.shape)
            clip_startings = np.arange(0,30,1.5)
            #print("clip_startings: shape:",clip_startings.shape,"content:",clip_startings)

            j=0 #for the filename
            
            for start in clip_startings:
                end = start + 3 #start and end point of the clip
                #print("clip number: {}, start: {}, end: {}".format(j,start,end))
                save_filename = './data/fma_small_raw_array_22050_overlapped/' + directory + "/" +  str(track_id) + "_" + str(j)
                j+=1 # for the filename
                
                start_sample = int(start*frame_rate)
                end_sample = int(end*frame_rate)
                #print("start_sample:",start_sample,"end_sample:",end_sample)
                if(start == 28.5):
                    #print("This is the last clip")
                    clip = np.append(pcm_audio[start_sample:], np.array([0 for i in range(int(1.5*frame_rate))], dtype=np.int16))
                    #print("padded clip: shape",clip.shape,"content:",clip)
                else:
                    clip = pcm_audio[start_sample:end_sample] #extract the clip
                #print("samples per clip:",len(clip),"duration in seconds = ",len(clip)/frame_rate)
                
                #print("Saving the stft vector in file:",save_filename)
                #print("array shape:",clip.shape,"dtype: ",clip.dtype)
                np.save(save_filename,clip)
                #print("shape of the stft vector (clip " + str(i//3) + "): ", stft.shape)
                
        except Exception as e: #skip the songs which give error (file corrupted or song too short)
            print("ERROR on file: ",filename,"\nError information:", e)
            #error_list.append(filename)
            
        progress_bar +=1
        if(len(error_list)>0):
            print("***ERROR: {} tracks caused errors: {}".format(len(error_list),error_list))
        #else:
            #print("Procedure completed without errors.")
            
    print("\n\n###Procedure completed with {} errors: {}".format(len(error_list),error_list))


In [86]:
generate_raw_array_files(train, 'train')

Generating files in the folder: train ...
File:  ./data/fma_small/000/000002.mp3 ,Progress:1/6400
File:  ./data/fma_small/000/000005.mp3 ,Progress:2/6400
File:  ./data/fma_small/000/000010.mp3 ,Progress:3/6400
File:  ./data/fma_small/000/000140.mp3 ,Progress:4/6400
File:  ./data/fma_small/000/000141.mp3 ,Progress:5/6400
File:  ./data/fma_small/000/000190.mp3 ,Progress:6/6400
File:  ./data/fma_small/000/000193.mp3 ,Progress:7/6400
File:  ./data/fma_small/000/000194.mp3 ,Progress:8/6400


KeyboardInterrupt: 

In [44]:
generate_raw_array_files(val, 'validation')

Generating files in the folder: validation ...
track id:  148
File: ./data/fma_small/000/000148.mp3
pcm_audio: shape: (661489,) content: [0 0 0 ... 0 0 0]
padding needed because:pcm_audio.shape:  (661489,)
padding: content: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] shape: (11,)
padded audio: shape:  (661500,)
pcm_audio: shape: (661500,)
clip_startings: shape: (20,) content: [ 0.   1.5  3.   4.5  6.   7.5  9.  10.5 12.  13.5 15.  16.5 18.  19.5
 21.  22.5 24.  25.5 27.  28.5]
clip number: 0, start: 0.0, end: 3.0
start_sample: 0 end_sample: 66150
samples per clip: 66150 duration in seconds =  3.0
Saving the stft vector in file: ./data/fma_small_raw_array/validation/148_0
clip number: 1, start: 1.5, end: 4.5
start_sample: 33075 end_sample: 99225
samples per clip: 66150 duration in seconds =  3.0
Saving the stft vector in file: ./data/fma_small_raw_array/validation/148_1
clip number: 2, start: 3.0, end: 6.0
start_sample: 66150 end_sample: 132300
samples per clip: 66150 duration in seconds =  3.0


pcm_audio: shape: (661489,) content: [0 0 0 ... 0 0 0]
padding needed because:pcm_audio.shape:  (661489,)
padding: content: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] shape: (11,)
padded audio: shape:  (661500,)
pcm_audio: shape: (661500,)
clip_startings: shape: (20,) content: [ 0.   1.5  3.   4.5  6.   7.5  9.  10.5 12.  13.5 15.  16.5 18.  19.5
 21.  22.5 24.  25.5 27.  28.5]
clip number: 0, start: 0.0, end: 3.0
start_sample: 0 end_sample: 66150
samples per clip: 66150 duration in seconds =  3.0
Saving the stft vector in file: ./data/fma_small_raw_array/validation/602_0
clip number: 1, start: 1.5, end: 4.5
start_sample: 33075 end_sample: 99225
samples per clip: 66150 duration in seconds =  3.0
Saving the stft vector in file: ./data/fma_small_raw_array/validation/602_1
clip number: 2, start: 3.0, end: 6.0
start_sample: 66150 end_sample: 132300
samples per clip: 66150 duration in seconds =  3.0
Saving the stft vector in file: ./data/fma_small_raw_array/validation/602_2
clip number: 3, start: 4

KeyboardInterrupt: 

In [40]:
generate_raw_array_files(test, 'test')

Generating files in the folder: test ...
track id:  182
File: ./data/fma_small/000/000182.mp3
Progress:1/800
Procedure completed without errors.
track id:  704
File: ./data/fma_small/000/000704.mp3
Progress:2/800
Procedure completed without errors.
track id:  705
File: ./data/fma_small/000/000705.mp3
Progress:3/800
Procedure completed without errors.
track id:  706
File: ./data/fma_small/000/000706.mp3
Progress:4/800
Procedure completed without errors.
track id:  707
File: ./data/fma_small/000/000707.mp3
Progress:5/800
Procedure completed without errors.
track id:  708
File: ./data/fma_small/000/000708.mp3
Progress:6/800
Procedure completed without errors.
track id:  709
File: ./data/fma_small/000/000709.mp3
Progress:7/800
Procedure completed without errors.
track id:  890
File: ./data/fma_small/000/000890.mp3
Progress:8/800
Procedure completed without errors.
track id:  892
File: ./data/fma_small/000/000892.mp3
Progress:9/800
Procedure completed without errors.
track id:  1066
File: .

Progress:78/800
Procedure completed without errors.
track id:  12059
File: ./data/fma_small/012/012059.mp3
Progress:79/800
Procedure completed without errors.
track id:  12060
File: ./data/fma_small/012/012060.mp3
Progress:80/800
Procedure completed without errors.
track id:  12061
File: ./data/fma_small/012/012061.mp3
Progress:81/800
Procedure completed without errors.
track id:  12062
File: ./data/fma_small/012/012062.mp3
Progress:82/800
Procedure completed without errors.
track id:  12064
File: ./data/fma_small/012/012064.mp3
Progress:83/800
Procedure completed without errors.
track id:  12065
File: ./data/fma_small/012/012065.mp3
Progress:84/800
Procedure completed without errors.
track id:  12066
File: ./data/fma_small/012/012066.mp3
Progress:85/800
Procedure completed without errors.
track id:  12067
File: ./data/fma_small/012/012067.mp3
Progress:86/800
Procedure completed without errors.
track id:  12179
File: ./data/fma_small/012/012179.mp3
Progress:87/800
Procedure completed w

Progress:155/800
Procedure completed without errors.
track id:  22001
File: ./data/fma_small/022/022001.mp3
Progress:156/800
Procedure completed without errors.
track id:  23010
File: ./data/fma_small/023/023010.mp3
Progress:157/800
Procedure completed without errors.
track id:  23041
File: ./data/fma_small/023/023041.mp3
Progress:158/800
Procedure completed without errors.
track id:  23063
File: ./data/fma_small/023/023063.mp3
Progress:159/800
Procedure completed without errors.
track id:  23862
File: ./data/fma_small/023/023862.mp3
Progress:160/800
Procedure completed without errors.
track id:  25055
File: ./data/fma_small/025/025055.mp3
Progress:161/800
Procedure completed without errors.
track id:  26859
File: ./data/fma_small/026/026859.mp3
Progress:162/800
Procedure completed without errors.
track id:  26861
File: ./data/fma_small/026/026861.mp3
Progress:163/800
Procedure completed without errors.
track id:  26902
File: ./data/fma_small/026/026902.mp3
Progress:164/800
Procedure c

Progress:231/800
Procedure completed without errors.
track id:  40230
File: ./data/fma_small/040/040230.mp3
Progress:232/800
Procedure completed without errors.
track id:  40231
File: ./data/fma_small/040/040231.mp3
Progress:233/800
Procedure completed without errors.
track id:  40232
File: ./data/fma_small/040/040232.mp3
Progress:234/800
Procedure completed without errors.
track id:  40233
File: ./data/fma_small/040/040233.mp3
Progress:235/800
Procedure completed without errors.
track id:  40234
File: ./data/fma_small/040/040234.mp3
Progress:236/800
Procedure completed without errors.
track id:  40235
File: ./data/fma_small/040/040235.mp3
Progress:237/800
Procedure completed without errors.
track id:  40236
File: ./data/fma_small/040/040236.mp3
Progress:238/800
Procedure completed without errors.
track id:  40237
File: ./data/fma_small/040/040237.mp3
Progress:239/800
Procedure completed without errors.
track id:  40238
File: ./data/fma_small/040/040238.mp3
Progress:240/800
Procedure c

Progress:307/800
Procedure completed without errors.
track id:  56895
File: ./data/fma_small/056/056895.mp3
Progress:308/800
Procedure completed without errors.
track id:  56929
File: ./data/fma_small/056/056929.mp3
Progress:309/800
Procedure completed without errors.
track id:  57271
File: ./data/fma_small/057/057271.mp3
Progress:310/800
Procedure completed without errors.
track id:  57272
File: ./data/fma_small/057/057272.mp3
Progress:311/800
Procedure completed without errors.
track id:  57273
File: ./data/fma_small/057/057273.mp3
Progress:312/800
Procedure completed without errors.
track id:  57274
File: ./data/fma_small/057/057274.mp3
Progress:313/800
Procedure completed without errors.
track id:  57540
File: ./data/fma_small/057/057540.mp3
Progress:314/800
Procedure completed without errors.
track id:  59562
File: ./data/fma_small/059/059562.mp3
Progress:315/800
Procedure completed without errors.
track id:  59581
File: ./data/fma_small/059/059581.mp3
Progress:316/800
Procedure c

...

Progress:459/800
Procedure completed without errors.
track id:  93523
File: ./data/fma_small/093/093523.mp3
Progress:460/800
Procedure completed without errors.
track id:  94093
File: ./data/fma_small/094/094093.mp3
Progress:461/800
Procedure completed without errors.
track id:  94099
File: ./data/fma_small/094/094099.mp3
Progress:462/800
Procedure completed without errors.
track id:  94101
File: ./data/fma_small/094/094101.mp3
Progress:463/800
Procedure completed without errors.
track id:  94102
File: ./data/fma_small/094/094102.mp3
Progress:464/800
Procedure completed without errors.
track id:  94803
File: ./data/fma_small/094/094803.mp3
Progress:465/800
Procedure completed without errors.
track id:  95486
File: ./data/fma_small/095/095486.mp3
Progress:466/800
Procedure completed without errors.
track id:  95535
File: ./data/fma_small/095/095535.mp3
Progress:467/800
Procedure completed without errors.
track id:  97286
File: ./data/fma_small/097/097286.mp3
Progress:468/800
Procedure c

Progress:535/800
Procedure completed without errors.
track id:  110774
File: ./data/fma_small/110/110774.mp3
Progress:536/800
Procedure completed without errors.
track id:  110775
File: ./data/fma_small/110/110775.mp3
Progress:537/800
Procedure completed without errors.
track id:  110776
File: ./data/fma_small/110/110776.mp3
Progress:538/800
Procedure completed without errors.
track id:  110777
File: ./data/fma_small/110/110777.mp3
Progress:539/800
Procedure completed without errors.
track id:  110778
File: ./data/fma_small/110/110778.mp3
Progress:540/800
Procedure completed without errors.
track id:  110779
File: ./data/fma_small/110/110779.mp3
Progress:541/800
Procedure completed without errors.
track id:  111146
File: ./data/fma_small/111/111146.mp3
Progress:542/800
Procedure completed without errors.
track id:  111147
File: ./data/fma_small/111/111147.mp3
Progress:543/800
Procedure completed without errors.
track id:  111148
File: ./data/fma_small/111/111148.mp3
Progress:544/800
Pr

Progress:611/800
Procedure completed without errors.
track id:  119828
File: ./data/fma_small/119/119828.mp3
Progress:612/800
Procedure completed without errors.
track id:  119830
File: ./data/fma_small/119/119830.mp3
Progress:613/800
Procedure completed without errors.
track id:  119831
File: ./data/fma_small/119/119831.mp3
Progress:614/800
Procedure completed without errors.
track id:  120206
File: ./data/fma_small/120/120206.mp3
Progress:615/800
Procedure completed without errors.
track id:  121415
File: ./data/fma_small/121/121415.mp3
Progress:616/800
Procedure completed without errors.
track id:  121595
File: ./data/fma_small/121/121595.mp3
Progress:617/800
Procedure completed without errors.
track id:  121913
File: ./data/fma_small/121/121913.mp3
Progress:618/800
Procedure completed without errors.
track id:  121915
File: ./data/fma_small/121/121915.mp3
Progress:619/800
Procedure completed without errors.
track id:  121922
File: ./data/fma_small/121/121922.mp3
Progress:620/800
Pr

Progress:687/800
Procedure completed without errors.
track id:  134793
File: ./data/fma_small/134/134793.mp3
Progress:688/800
Procedure completed without errors.
track id:  134794
File: ./data/fma_small/134/134794.mp3
Progress:689/800
Procedure completed without errors.
track id:  134795
File: ./data/fma_small/134/134795.mp3
Progress:690/800
Procedure completed without errors.
track id:  134796
File: ./data/fma_small/134/134796.mp3
Progress:691/800
Procedure completed without errors.
track id:  134797
File: ./data/fma_small/134/134797.mp3
Progress:692/800
Procedure completed without errors.
track id:  136054
File: ./data/fma_small/136/136054.mp3
Progress:693/800
Procedure completed without errors.
track id:  137605
File: ./data/fma_small/137/137605.mp3
Progress:694/800
Procedure completed without errors.
track id:  137624
File: ./data/fma_small/137/137624.mp3
Progress:695/800
Procedure completed without errors.
track id:  137627
File: ./data/fma_small/137/137627.mp3
Progress:696/800
Pr

Progress:763/800
Procedure completed without errors.
track id:  145762
File: ./data/fma_small/145/145762.mp3
Progress:764/800
Procedure completed without errors.
track id:  145777
File: ./data/fma_small/145/145777.mp3
Progress:765/800
Procedure completed without errors.
track id:  145887
File: ./data/fma_small/145/145887.mp3
Progress:766/800
Procedure completed without errors.
track id:  147020
File: ./data/fma_small/147/147020.mp3
Progress:767/800
Procedure completed without errors.
track id:  147021
File: ./data/fma_small/147/147021.mp3
Progress:768/800
Procedure completed without errors.
track id:  147022
File: ./data/fma_small/147/147022.mp3
Progress:769/800
Procedure completed without errors.
track id:  147023
File: ./data/fma_small/147/147023.mp3
Progress:770/800
Procedure completed without errors.
track id:  147024
File: ./data/fma_small/147/147024.mp3
Progress:771/800
Procedure completed without errors.
track id:  147261
File: ./data/fma_small/147/147261.mp3
Progress:772/800
Pr